# Outbound Auth

Outbound Auth allows agents and the AgentCore Gateway to securely access AWS resources and third-party services on behalf of users who have been authenticated and authorized during Inbound Auth. To integrate authorization with an AWS resource or third-party service, it's necessary to configure both Inbound Auth and Outbound Auth.

With just-enough access and secure permission delegation supported by AgentCore Identity, agents can seamlessly and securely access AWS resources and third-party tools such as GitHub, Google, Salesforce, and Slack. Agents can perform actions on these services either on behalf of users or independently, provided there is pre-authorized user consent. Additionally, you can reduce consent fatigue using a secure token vault and create streamlined AI agent experiences.

## Outbound Authentication Configuration

First, you register your client application with third-party providers and then create an Outbound Auth. You specify how you want to validate access to the AWS resource or third-party service or AgentCore Gateway targets. You can use OAuth 2LO/3LO or API keys. With OAuth, you can select from providers that AgentCore Identity provides. In which case you enter the configuration details for the providers from AgentCore Identity. Alternatively, you can supply details for a custom provider. 

When a user wants access to an AWS resource or third-party service or AgentCore Gateway target, the Outbound Auth confirms that the access tokens provided by Incoming Auth are valid and if so, allows access to the resource.

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>

## Resource credential providers

This is a component that agent code uses to retrieve credentials of downstream resource servers (e.g., Google, GitHub) to access them, e.g., fetch emails from Gmail, add a meeting to Google Calendar.  It removes the heavy-lifting of agent developers implementing 2LO and 3LO OAuth2 orchestration flows across end-users, agent code, and external authorization servers. AgentCore provides both a custom OAuth2 credential provider and a list of built-in providers such Google, GitHub, Slack, Salesforce with authorization server endpoint and provider-specific parameters pre-filled.
  

Bedrock AgentCore Identity provides OAuth2 and API Key Credential Providers for agent developers to authenticate with external resources that support OAuth2 or API key. In the following example, we will walk you through configuring an API Key credential provider.  An agent can then use the API Key credential provider to retrieve the API key for any agent operations. Please refer to the documentation for the other credential providers.

 ### Creating a resource credential provider.

 Here is an example of creating an OAuth2 resource credential provider. In this example we are creating a credential provider for "Google". This is one of the named resource credential providers. You can also configure this credential provider using the AgentCore console.

```
from bedrock_agentcore.services.identity import IdentityClient

identity_client = IdentityClient("us-east-1")

# Configure Google OAuth2 provider - On-Behalf-Of User
google_provider = identity_client.create_oauth2_credential_provider({
    "name":"<name-of-your-cred-provider>", 
    "credentialProviderVendor":"GoogleOauth2",
    "oauth2ProviderConfigInput":{
        "googleOauth2ProviderConfig": {
            "clientId": "<your-google-client-id>",
            "clientSecret": "<your-google-client-secret>"
        }
    }
})
print(google_provider)
```

 ### Retrieving OAuth2 access tokens with the Resource credential provider.

Here is an example of retrieving the OAuth2 access tokens from the credential provider. The agent can then use the access token to interact with resources protected by OAuth. e.g: in our example , this is Google drive data and the scope is readonly. In order to retrieve credentials like access_token from the credential provider, you can decorate your function as shown below.

```
import asyncio
from bedrock_agentcore.identity.auth import requires_access_token, requires_api_key

# This annotation helps agent developer to obtain access tokens from external applications
@requires_access_token(
    provider_name="<name-of-your-cred-provider>",
    scopes=["https://www.googleapis.com/auth/drive.metadata.readonly"], # Google OAuth2 scopes
    auth_flow="USER_FEDERATION", # On-behalf-of user (3LO) flow
    on_auth_url=lambda x: print("\nPlease copy and paste this URL in your browser:\n" + x), # prints authorization URL to console
    force_authentication=True,
)
async def write_to_google_drive(*, access_token: str):
    print(access_token)

asyncio.run(write_to_google_drive(access_token=""))
```

Here are the various parameters you can use with the @require_access_token decorator.


| Parameter Name      | Description                                                              |
|:--------------------|:-------------------------------------------------------------------------|
| provider_name       | The credential provider name                                             |
| into                | Parameter name to inject the token into                                  |
| scopes              | OAuth2 scopes to request                                                 |
| on_auth_url	      | Callback for handling authorization URLs                                 |
| auth_flow           | Authentication flow type ("M2M" or "USER_FEDERATION")                    |
| callback_url        | OAuth2 callback URL                                                      |
| force_authentication| Force re-authentication                                                  |
| token_poller        | Custom token poller implementation                                       |

		


# Hosting Strands Agents in Amazon Bedrock AgentCore Runtime

## Overview


In this tutorial we will develop a scheduling agent using Strands agents that can list the events from the users Google Calendar. We will configure a credential provider to help with credential management with Google. We can use the named provider for Google and modify our agent code to call the credential provider and use the access_token to get the users calendar events or schedule from Google.

### Tutorial Architecture

<div style="text-align:center">
    <img src="images/outbound_auth_3lo.png" width="90%"/>
</div>


### Tutorial Details

| Information         | Details                                                                  |
|:--------------------|:-------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                           |
| Agent type          | Single                                                                   |
| Agentic Framework   | Strands Agents                                                           |
| LLM model           | Anthropic Claude Sonnet 3.7                                              |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Claude Model |
| Tutorial vertical   | Cross-vertical                                                           |
| Example complexity  | Medium                                                                   |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                             |
| Credential Provider | Type : OAuth2 - Google Provider                                          |


### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Claude models
* Using Strands Agents
* Using AgentCore egress Auth with OAuth2 Google credential provider.


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

##  Configure Inbound Auth with Cognito as the IDP
Lets provision a Cognito Userpool with an App client and one test user. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server. To do so, we will use the `setup_cognito_user_pool` support function from our `utils` script.

Note: The Cognito access_token is valid for 2 hours only. If the access_token expires you can vend another access_token by using the `reauthenticate_user` method.

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool, reauthenticate_user

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")

## Configure Google for OAuth2. ( On behalf of user flow )
Follow these steps to register your app, create a project, and configure OAuth credentials for Google Drive access:

In this section, we will configure your Google to access the Google Calendar API with readonly permissions.

1. Create a Project in Google Developer Console
    1.    Go to the [Google Developer Console](https://console.developers.google.com/)
    2.    In the top navigation bar, click on “Create Project”
    3.    Enter a Project Name.
    4.    Choose an Organization or leave as “No organization” if not applicable.
    5.    Click Create. Your new project will appear in the project list.
2. Enable Google Calendar API
    1.    With your project selected ( using the checkbox),  open the left menu (hamburger menu) and go to APIs & Services > Library.
    2.    In the search bar, type Google Calendar API.
    3.    Click on Google Calendar API in the results, then click Enable.
3. Configure OAuth Consent Screen
    1.    In the left menu, go to APIs & Services > OAuth consent screen.
    2.    Click “Get started”
    3.    Fill in the required fields:
    4.    App Name
    5.    User Support Email
    6. Click Next and then select the right Audience type, i.e. Internal or External, Click Next, ( If you select External, then ensure you have entered email ids of users entered here, so you can test with those users)
    7.    Developer Contact Information, Enter your email id.
    8.  Click “Finish” after accepting the terms and conditions. an then click “Create”.
4. Add your google email id as a test user
    1.    In the left menu, go to APIs & Services > OAuth consent screen.
    2.    Select "Audience" from the left hand menu.
    3.    Under Test users, click "+ Add Users" and add your gmail id for the Google account.
5. Create OAuth 2.0 Credentials
    1.    Go to APIs & Services from the left hand menu and then select  > Credentials.
    2.    Click Create Credentials > OAuth client ID.
    3.    Select Web application as the application type.
    4.    Enter a name for the credentials.
    5.    Under Authorized redirect URIs, add your app’s redirect URI. You can find the redirect URI from the [AWS Docs](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity-getting-started-prerequisites.html)<br>
        The redirect URI is : https://bedrock-agentcore.us-east-1.amazonaws.com/identities/oauth2/callback
    6.    Click Create.
6. Obtain Client ID and Client Secret
    1.    After creation, a dialog will display your Client ID and Client Secret. Copy these for later use.
    2.    Download the credentials as a JSON file or copy them for use in your app’s configuration.
7. Update the Data access 
    1. Go to APIs & Services from the left hand menu and then select  > Credentials.
    2. Select the “web app” you created in step 4d.
    3. Select “Data access” on the left hand menu
    4. Click “Add or remove scopes”
    5. Add scopes based on your use case. E.g: For Google drive add, “https://www.googleapis.com/auth/calendar.readonly” under “Manually add scopes” and click update followed by Save/Update.
    6. Click “Save” again on the “Data access” page to save your configuration.
8. Use the Credentials in Your Agent
    1.    In the next section, we will configure a resource credentials provider to use the Client ID, Client Secret, and Redirect URI for the OAuth 3-legged flow.

#### Configure the Google OAuth2 credential provider.

Modify the following code and replace the following :
1. <your-google-client-id> with the "client id" recorded from Step 6.1 above
2. <your-google-client-secret> with the "client secret" recorded from Step 6.2 above.

Important : Please ensure you replace the entire place holder string including the "<" and ">"


Once the client id and client secret are updated, run the below code to create a credentials provider for Google. <br>
Resource credential providers in AgentCore Identity act as intelligent intermediaries that manage the complex relationships between agents, identity providers, and resource servers. Each provider encapsulates the specific endpoint configuration required for a particular service or identity system. The service provides built-in providers for popular services including Google, GitHub, Slack, and Salesforce, with authorization server endpoints and provider-specific parameters pre-configured to reduce development effort. AgentCore Identity supports custom configurations through configurable OAuth2 credential providers that can be tailored to work with any OAuth2-compatible resource server.



In [ ]:
from bedrock_agentcore.services.identity import IdentityClient

identity_client = IdentityClient(region)

# Configure Google OAuth2 provider - On-Behalf-Of User
google_provider = identity_client.create_oauth2_credential_provider({
    "name":"google-cal-provider",
    "credentialProviderVendor":"GoogleOauth",
    "oauth2ProviderConfigInput":{
        "googleOauth2ProviderConfig": {
            "clientId": "<client-id>",
            "clientSecret": "<client-secret>"
        }
    }
})
print(google_provider)

## Preparing your agent for deployment on AgentCore Runtime

### Strands Agent with a model hosted on Amazon Bedrock
Here is a Strands agent code that includes the following :
1. Creates a new tool called "get_calendar_events_today", to get the events from your Google Calendar for today
2. Uses the Credential provider created in the previous step to fetch the access_token from Google. This includes the user consent flow where the consent user is sent to the user for approval as part of the 3LO flow.
3. The Strands agent calls the tool for any user requests related to the users agenda.

In [ ]:
%%writefile strands_claude_google_3lo.py
import os
import datetime  
import json
import asyncio
import traceback

from strands import Agent
from strands import tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

os.environ["STRANDS_OTEL_ENABLE_CONSOLE_EXPORT"] = "true"
os.environ["OTEL_PYTHON_EXCLUDED_URLS"] = "/ping,/invocations"

# Required OAuth2 scope for Google Calendar API
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

google_access_token = None  # Global variable to store the access token

@tool(name="Get_calendar_events_today", description="Retrieves the calendar events for the day from your Google Calendar")
def get_calendar_events_today() -> str:
    global google_access_token
    
    # Check if we already have a token
    if not google_access_token:
        return json.dumps({"auth_required": True, "message": "Google Calendar authentication is required. Please wait while we set up the authorization.", "events": []})
    
    # Create credentials from the provided access token
    creds = Credentials(token=google_access_token, scopes=SCOPES)
    try:
        service = build("calendar", "v3", credentials=creds)
        # Call the Calendar API
        today_start = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        today_end = today_start.replace(hour=23, minute=59, second=59)
        
        # Format with CDT timezone (-05:00)
        timeMin = today_start.strftime('%Y-%m-%dT00:00:00-05:00')
        timeMax = today_end.strftime('%Y-%m-%dT23:59:59-05:00')
        
        events_result = (
            service.events()
            .list(
                calendarId="primary",
                timeMin=timeMin,
                timeMax=timeMax,
                singleEvents=True,
                orderBy="startTime",
            )
            .execute()
        )
        events = events_result.get("items", [])

        if not events:
            return json.dumps({"events": []})  # Return empty events array as JSON

        return json.dumps({"events": events})  # Return events wrapped in an object
    except HttpError as error:
        error_message = str(error)
        return json.dumps({"error": error_message, "events": []})
    except Exception as e:
        error_message = str(e)
        return json.dumps({"error": error_message, "events": []})


# Initialize the agent with tools
agent = Agent(tools=[get_calendar_events_today])

# Initialize app and streaming queue
app = BedrockAgentCoreApp()

class StreamingQueue:
    def __init__(self):
        self.finished = False
        self.queue = asyncio.Queue()
        
    async def put(self, item):
        await self.queue.put(item)

    async def finish(self):
        self.finished = True
        await self.queue.put(None)

    async def stream(self):
        while True:
            item = await self.queue.get()
            if item is None and self.finished:
                break
            yield item

queue = StreamingQueue()


async def on_auth_url(url: str):
    print(f"Authorization url: {url}")
    await queue.put(f"Authorization url: {url}")


async def agent_task(user_message: str):
    try:
        await queue.put("Begin agent execution")
        
        # Call the agent first to see if it needs authentication
        response = agent(user_message)
        
        # Extract text content from the response structure
        response_text = ""
        if isinstance(response.message, dict):
            content = response.message.get('content', [])
            if isinstance(content, list):
                for item in content:
                    if isinstance(item, dict) and 'text' in item:
                        response_text += item['text']
        else:
            response_text = str(response.message)
        
        # Check if the response indicates authentication is required
        # Look for various keywords that indicate authentication issues
        auth_keywords = [
            "authentication", "authorize", "authorization", "auth", 
            "sign in", "login", "access", "permission", "credential",
            "need authentication", "requires authentication"
        ]
        needs_auth = any(keyword.lower() in response_text.lower() for keyword in auth_keywords)
       
        if needs_auth:
            await queue.put("Authentication required for Google Calendar access. Starting authorization flow...")
            
            # Trigger the 3LO authentication flow
            try:
                global google_access_token
                google_access_token = await need_token_3LO_async(access_token='')
                await queue.put("Authentication successful! Retrying calendar request...")
                
                # Retry the agent call now that we have authentication
                response = agent(user_message)
            except Exception as auth_error:
                # print("Exception occurred:")
                # traceback.print_exc()
                print(f"auth_error:", auth_error)
                await queue.put(f"Authentication failed: {str(auth_error)}")
        
        await queue.put(response.message)
        await queue.put("End agent execution")
    except Exception as e:
        await queue.put(f"Error: {str(e)}")
    finally:
        await queue.finish()



@requires_access_token(
    provider_name="google-cal-provider",
    scopes=SCOPES,
    auth_flow='USER_FEDERATION',
    on_auth_url=on_auth_url,
    force_authentication=True,
)
async def need_token_3LO_async(*, access_token: str):
    global google_access_token
    google_access_token = access_token  # Update the global access token
    return access_token

@app.entrypoint
async def agent_invocation(payload):
    user_message = payload.get("prompt", "No prompt found in input, please guide customer to create a json payload with prompt key")
    
    # Create and start the agent task
    task = asyncio.create_task(agent_task(user_message))
    
    # Return the stream, but ensure the task runs concurrently
    async def stream_with_task():
        # Stream results as they come
        async for item in queue.stream():
            yield item
        
        # Ensure the task completes
        await task
    
    return stream_with_task()


if __name__ == "__main__":
    app.run()


## Deploying the agent to AgentCore Runtime
The CreateAgentRuntime operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent.

Note: Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

Note : The authorizer_configuration is configured for Inbound Auth with Cognito.

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

discovery_url = cognito_config.get("discovery_url")
client_id = cognito_config.get("client_id")
agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_google_3lo.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_agent_google_3lo",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
response

## Review the AgentCore configuration

In [ ]:
!cat .bedrock_agentcore.yaml

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### Add extra required policies to auto-created role
Since we are adding some outbound identity to our agent, we will need to get some API Keys and Secrets that are not available in the auto-created role. To do so, we will need to add some extra permissions to our auto-created IAM role. Let's first get this role and then add those permissions to it.

In [ ]:
import json
import boto3
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "GetResourceAPIKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey",
                "bedrock-agentcore:GetResourceOauth2Token"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SecretManager",
            "Effect": "Allow",
            "Action": [
                "secretsmanager:GetSecretValue"
            ],
            "Resource": "*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

You will notice that the agent calls the "Get_calendar_events_today" tool and triggers the 3 Legged Outh flow. You will be presented with the Authorization Url. Click on the Authorization url Or copy/paste it in a new browser session/tab to complete the user consent flow.
Once the Authorization completes, The credential provider "google-cal-provider3" will fetch the access_token from Google and complete the tool execution to fetch the events from your calendar.

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
bearer_token = reauthenticate_user(cognito_config.get("client_id"))

invoke_response = agentcore_runtime.invoke(
    {"prompt": "What is in my agenda for today?"}, 
    bearer_token=bearer_token
)
invoke_response

## Optional - Test the agent using a Streamlit App

You can test your deployed agent using a Streamlit web application that provides a nice chat interface. The `chatbot_app_cognito.py` file in this directory creates a web-based chatbot that:

- Automatically reads configuration from `.bedrock_agentcore.yaml`
- Provides Cognito authentication
- Shows a modern chat interface with streaming responses
- Handles the 3LO OAuth flow for Google Calendar access

### Running the Streamlit App

You can run the Streamlit app in several ways:

#### Option 1: Run from Jupyter Notebook (Current Directory)
- Run the cell below to start the Streamlit app directly from this notebook:
- Login: Use the credentials testuser / TempPassword123! (the default test user created by the Cognito setup)
- Test some simple prompts like "Tell me a joke"
- Test with a prompt that will trigger the `Get_calendar_events_today` tool like "What is on my agenda for today?".
- You will see the Authorization url returned. Click on the url or copy/paste the url to a new browser tab/window to complete the user consent flow.

In [ ]:
import subprocess
import sys
import os

# Change to the current directory where the chatbot_app_cognito.py file is located
notebook_dir = os.getcwd()
print(f"Starting Streamlit app from: {notebook_dir}")

# Start the Streamlit app
# Note: This will run in the background, and you can access it at http://localhost:8501
print("Starting Streamlit app...")
print("Once started, open your browser and go to: http://localhost:8501")
print("To stop the app, interrupt the kernel or press Ctrl+C in the terminal")

try:
    # Run streamlit in the current directory
    process = subprocess.Popen([sys.executable, "-m", "streamlit", "run", "chatbot_app_cognito.py", "--server.port=8501"], 
                              cwd=notebook_dir,
                              stdout=subprocess.PIPE, 
                              stderr=subprocess.STDOUT, 
                              text=True)
    
    # Print the output as it comes
    for line in iter(process.stdout.readline, ''):
        if line:
            print(line.rstrip())
        if "Local URL: http://localhost:8501" in line:
            print("\n🎉 Streamlit app is ready!")
            print("Open your browser and visit: http://localhost:8501")
            break
            
except KeyboardInterrupt:
    print("\nStreamlit app stopped.")
    process.terminate()
except Exception as e:
    print(f"Error starting Streamlit app: {e}")

#### Option 2: Run from Terminal

Alternatively, you can run the Streamlit app from your terminal:

```bash
# Navigate to the current directory
cd 01-tutorials/03-AgentCore-identity/05-Outbound_Auth_3lo/

# Run the Streamlit app
streamlit run chatbot_app_cognito.py
```

#### Using the Streamlit App

1. **Login**: Use the credentials `testuser` / `TempPassword123!` (the default test user created by the Cognito setup)
2. **Chat**: Ask questions like "What is in my agenda for today?"
3. **OAuth Flow**: When the agent needs Google Calendar access, you'll see an authorization URL - click it to complete the OAuth flow
4. **Features**: The app includes:
   - Real-time streaming responses
   - Clickable URLs
   - Modern chat interface
   - Context awareness
   - Error handling with informative messages

The app automatically reads all configuration from your `.bedrock_agentcore.yaml` file, so it will use the same agent runtime you just deployed.

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

## Congratulations!